In [ ]:
!pip install llvmlite --ignore-installed
!pip install sktime
!pip install prophet
!pip install pystan==2.19.1.1
!pip install fbprophet==0.7.1

In [1]:
import pandas as pd
import os
import sktime
import numpy as np
import warnings
import itertools
from tqdm import tqdm

from sktime.utils.plotting import plot_series
from sktime.forecasting.model_selection import temporal_train_test_split

import fbprophet
from fbprophet import Prophet
from fbprophet.plot import plot_plotly, plot_components_plotly, plot_yearly, plot_weekly, plot_cross_validation_metric
from fbprophet.diagnostics import cross_validation, performance_metrics

warnings.filterwarnings("ignore")

In [2]:
path_root = "/home/jovyan/hfactory_magic_folders/natixis_data_challenge_22_23/it_porto/Data/extraction"
path_cockpit = path_root + "/cockpit_20221221.csv"
path_items_info = path_root + "/item_info_20221221.csv"
path_items_trend = path_root + "/item_trend_20221221.csv"
path_zabbix = path_root + "/tmp_hosts_zabbix_20221221.csv"
path_mycloud =  path_root + "/mycloud_20221221.csv"

cockpit = pd.read_csv(path_cockpit,index_col=0)
items_info = pd.read_csv(path_items_info,index_col=0)
items_trend = pd.read_csv(path_items_trend,index_col=0)
zabbix = pd.read_csv(path_zabbix,index_col=0)
mycloud = pd.read_csv(path_mycloud,index_col=0)

In [3]:
def preprocess(y):
    y.reset_index()
    y = y.rename(columns={'clock': 'ds', 'value_max':'y'})
    y['ds'] = pd.to_datetime(y['ds'])
    return(y)
    
def get_split(y, splitsize=0.3):
    train, valid = temporal_train_test_split(y, test_size=0.3)
    return(train, valid)

In [4]:
def forecast(model, train, valid):
    """Fits a Prophet model to training data and makes forecasts on validation data.

    Parameters
    ----------
    model : Prophet
        Prophet model instantiated with the desired set of parameters.
    train : pd.DataFrame
        Training data, containing at least a date column (ds) and a target column (y).
    valid : pd.DataFrame
        Validation data, containing at least a date column (ds) and a target column (y).

    Returns
    -------
    pd.DataFrame
        Dataframe containing forecasts and ground truth on both training and validation data.
    """
    
    model.fit(train)
    full_df = pd.concat([train, valid], axis=0)
    pred = model.predict(full_df)
    pred['y'] = np.array(full_df['y'])
    return pred

def evaluate(pred, valid):
    """Evaluates the performance of Prophet forecasts on validation data.

    Parameters
    ----------
    pred : pd.DataFrame
        Dataframe containing forecasts and ground truth on both training and validation data.
    valid : pd.DataFrame
        Validation data, containing at least a date column (ds) and a target column (y).

    Returns
    -------
    dict
        Performance on validation data (RMSE and MAPE).
    """
    y_pred, y_true = np.array(pred[-len(valid):].yhat), np.array(pred[-len(valid):].y)
    mask = y_true != 0
    mape = np.mean(np.abs((y_true - y_pred) / y_true)[mask])
    rmse = np.sqrt(((y_true - y_pred) ** 2)[mask].mean())
    return {'RMSE': rmse, 'MAPE': mape}

# Cross validation

In [5]:
grid_base = {'changepoint_prior_scale': [0.001, 0.1, 10]# List of values to test for changepoint_prior_scale ,
        ,'changepoint_range': [0.5, 0.75, 1] # List of values to test for changepoint_range
        , 'holidays_prior_scale' : [0.001, 0.1, 10]
        , 'weekly_seasonality' : [True, False]
        , 'daily_seasonality' : [True, False]
        }

def grid_search(grid = grid_base, train = None, valid = None):
    """Performs a grid search and shows the performance associated with all parameter combinations in the grid.

    Parameters
    ----------
    grid : dict
        Dictionary where keys are parameters names and values are lists of param values to be tested.
    """
    RMSEs = []
    combinations = [dict(zip(grid.keys(), i)) for i in itertools.product(*grid.values())]
    eval_df = pd.DataFrame(combinations)
    for params in combinations:
        model = Prophet(**params)
        pred = forecast(model, train, valid)
        RMSEs.append(evaluate(pred, valid)['RMSE'])
    eval_df['RMSE'] = RMSEs
    #plot_grid_search(eval_df, grid)
    return(eval_df)

In [21]:
def optimize_timeseries(y, train_size=365):
    y = preprocess(y)
    train, valid = temporal_train_test_split(y, train_size = train_size)
    eval_df = grid_search(grid_base, train=train, valid=valid)
    return(eval_df)

In [22]:
# Going with an example

In [ ]:
items_trend["clock"] = pd.to_datetime(items_trend["clock"])
#items_trend['rank'] = items_trend.groupby(["itemid","item_type"])["clock"].rank(ascending=False)
#items_trend_clean = items_trend[items_trend["rank"] <= 365]

#y = items_trend_clean[items_trend_clean.itemid == 664016][["clock","value_max"]]
y = items_trend[items_trend.itemid == 664016][["clock","value_max"]]

optimize_timeseries(y)
# Uncomment to launch optimization

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.


Initial log joint probability = -15.5
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       455.013   3.50767e-07       5029.09      0.5068      0.5068      134   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     126       455.025   7.05127e-09       4640.32       0.296      0.6798      165   
Optimization terminated normally: 
  Convergence detected: absolute parameter change was below tolerance


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.


Initial log joint probability = -15.5
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       436.319    7.9814e-07       5196.22      0.5231      0.5231      136   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     141       436.358   1.10861e-06       4750.82   2.225e-10       0.001      230  LS failed, Hessian reset 
     163       436.363   8.02836e-09       4381.22      0.5545      0.5545      255   
Optimization terminated normally: 
  Convergence detected: absolute parameter change was below tolerance


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.


...

Initial log joint probability = -15.5
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       633.056     0.0189914        19.626           1           1      113   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       637.759     0.0189406       20.3916           1           1      224   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299        640.33     0.0159373       14.9268           1           1      337   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399        641.15    0.00188609       6.44544           1           1      461   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     499       641.705     0.0140747       16.9241           1           1      577   
    Iter      log prob        ||dx||      ||grad||       alpha     

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.


Initial log joint probability = -15.5
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       372.896    0.00980677        8.3563           1           1      122   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       375.327    0.00399585      0.846195           1           1      243   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       375.632      0.010725       4.23588           1           1      363   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399       375.821    0.00335393       2.26833      0.8864      0.8864      490   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     499       375.889    0.00471364       1.61092           1           1      618   
    Iter      log prob        ||dx||      ||grad||       alpha     

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.


Initial log joint probability = -15.5
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       374.303    0.00908254       11.2097       0.162           1      129   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       375.173     0.0213659       3.54598           1           1      248   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       375.568    0.00665461       8.40643           1           1      366   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     341       375.645   9.92752e-05      0.692727   2.789e-05       0.001      455  LS failed, Hessian reset 
     399       375.742     0.0288614        10.718           1           1      525   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     499       375.927     0.0237828       

In [15]:
# Comparing to baseline

In [18]:
model_baseline = Prophet()
z = preprocess(y)
train, valid = temporal_train_test_split(z, train_size = 365)
pred = forecast(model_baseline, train, valid)
dic_res = evaluate(pred, valid)
dic_res

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Initial log joint probability = -15.5
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      63       560.099    0.00370998       65.5353   4.535e-05       0.001      126  LS failed, Hessian reset 
      99       560.246   1.16768e-06       52.2051     0.07023           1      185   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     114       560.251   4.24108e-05         58.49    5.44e-07       0.001      254  LS failed, Hessian reset 
     141       560.254   1.15393e-06       64.1925   1.734e-08       0.001      330  LS failed, Hessian reset 
     157       560.254   8.75584e-09       75.7982      0.2201      0.2201      354   
Optimization terminated normally: 
  Convergence detected: absolute parameter change was below tolerance


{'RMSE': 8.941431104905725, 'MAPE': 0.21126690247781202}

In [ ]:
# Not a lot of interest

# Grid search CV

In [10]:
def grid_search_cv(grid = grid_base, initial = "365 days", period = "30 days", horizon = "30 days", y=y):
    """Plots the cross-validated performance of Prophet models trained with parameters in the grid.

    Parameters
    ----------
    grid : dict
        Dictionary where keys are parameters names and values are lists of param values to be tested.
    initial : str
        Number of days in the training set of each fold.
    period : str
        Offset between each fold (in number of days).
    horizon : str
        Number of days in the validation set of each fold.
    y : pd.DataFrame
        Whole dataset.
    """
    df = preprocess(y)
    RMSEs = []
    combinations = [dict(zip(grid.keys(), i)) for i in itertools.product(*grid.values())]
    eval_df = pd.DataFrame(combinations)
    for params in tqdm(combinations):
        model = Prophet(**params).fit(df)
        df_cv = cross_validation(model, initial=initial, period=period, horizon=horizon, parallel="threads")
        df_p = performance_metrics(df_cv, rolling_window=1)
        RMSEs.append(df_p['rmse'].values[0])
    eval_df['RMSE'] = RMSEs
    plot_grid_search(eval_df, grid)
    return(eval_df)

In [12]:
#grid_search_cv()
# Uncomment for launching the grid search cv, however it will take over 15min and as we saw earlier, it has limited interest

In [ ]:
y